In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '2'
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM, TextStreamer
from peft import LoraConfig, get_peft_model, AutoPeftModelForCausalLM, PeftModel
import torch

In [3]:
tokenizer = AutoTokenizer.from_pretrained('unsloth/Llama-3.2-3B-Instruct')

In [4]:
streamer = TextStreamer(tokenizer)

In [5]:
ori_model = AutoModelForCausalLM.from_pretrained(
    'unsloth/Llama-3.2-3B-Instruct', tie_word_embeddings=False, torch_dtype = torch.bfloat16,
).cuda()

Some weights of LlamaForCausalLM were not initialized from the model checkpoint at unsloth/Llama-3.2-3B-Instruct and are newly initialized: ['lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
ori_model.lm_head.weight.data = ori_model.model.embed_tokens.weight.data.clone()

In [7]:
state_dict = ori_model.state_dict()

In [8]:
from transformers.trainer_utils import get_last_checkpoint

latest = get_last_checkpoint("lora-embedding-256-llama3.2-3b-small-malaysian-reasoning")
latest

'lora-embedding-256-llama3.2-3b-small-malaysian-reasoning/checkpoint-2400'

In [9]:
from safetensors import safe_open

f = safe_open(f"{latest}/adapter_model.safetensors", framework="pt", device=0)
keys = f.keys()
keys = sorted(list(set([k.split('.lora')[0] for k in keys if '.lora' in k])))

In [10]:
from tqdm import tqdm

for k in tqdm(keys):
    k_ori = k.replace('base_model.model.', '') + '.weight'
    if 'embed_tokens' in k:
        post_A = '.lora_embedding_A'
        post_B = '.lora_embedding_B'
    else:
        post_A = '.lora_A.weight'
        post_B = '.lora_B.weight'
    A = k + post_A
    B = k + post_B
    
    W = state_dict[k_ori]
    if 'embed_tokens' not in k:
        W = W.t()
        
    A = f.get_tensor(A).type(W.dtype)
    B = f.get_tensor(B).type(W.dtype)
    with torch.no_grad():
        W.addmm_(A.t(), B.t(), alpha = 2)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████| 198/198 [00:03<00:00, 59.69it/s]


In [11]:
d = [
    {'role': 'user', 'content': 'ape isu dengan najib'}
]

inputs = tokenizer.apply_chat_template(d, return_tensors = 'pt').to('cuda')
generate_kwargs = dict(
    input_ids=inputs,
    max_new_tokens=50,
    top_p=0.95,
    top_k=50,
    temperature=0.0,
    do_sample=False,
    repetition_penalty=1.05,
    streamer=streamer
)
generation_output = ori_model.generate(**generate_kwargs)

/home/husein/ssd3/llama-flex-attention-multipack/src/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/husein/ssd3/llama-flex-attention-multipack/src/transformers/generation/configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a conseque

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 05 Jan 2025

<|eot_id|><|start_header_id|>user<|end_header_id|>

ape isu dengan najib<|eot_id|><|start_header_id|>assistant<|end_header_id|>

Najib Razak, yang berkhidmat sebagai Perdana Menteri Malaysia dari 2009 hingga 2018, telah menjadi subjek beberapa kontroversi dan skandal di Malaysia. Berikut adalah beberapa


In [12]:
d = [
    {'role': 'user', 'content': 'camne nk code react, reply in jawi'}
]

inputs = tokenizer.apply_chat_template(d, return_tensors = 'pt').to('cuda')
generate_kwargs = dict(
    input_ids=inputs,
    max_new_tokens=50,
    top_p=0.95,
    top_k=50,
    temperature=0.9,
    do_sample=False,
    repetition_penalty=1.1,
    streamer=streamer
)
generation_output = ori_model.generate(**generate_kwargs)

/home/husein/ssd3/llama-flex-attention-multipack/src/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 05 Jan 2025

<|eot_id|><|start_header_id|>user<|end_header_id|>

camne nk code react, reply in jawi<|eot_id|><|start_header_id|>assistant<|end_header_id|>

اونتوق ممبوات کومڤونين React يڠ موده اونتوق منچيڤتا بوراڠ دڠن تاجوق دان ڤيليهن "


In [13]:
d = [
    {'role': 'user', 'content': 'camne nk code react, reply in sabah'}
]

inputs = tokenizer.apply_chat_template(d, return_tensors = 'pt').to('cuda')
generate_kwargs = dict(
    input_ids=inputs,
    max_new_tokens=50,
    top_p=0.95,
    top_k=50,
    temperature=0.9,
    do_sample=False,
    repetition_penalty=1.1,
    streamer=streamer
)
generation_output = ori_model.generate(**generate_kwargs)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 05 Jan 2025

<|eot_id|><|start_header_id|>user<|end_header_id|>

camne nk code react, reply in sabah<|eot_id|><|start_header_id|>assistant<|end_header_id|>

Bah, kunuk sia mau kasi tau macam mana mo bikin komponen React yang boleh pakai untuk tingu senarai barang. Kita guna hook `useState` sama `useEffect` dari


In [16]:
tokenizer.push_to_hub('mesolitica/Llama-3.2-3B-Malaysian-Reasoning')

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/mesolitica/Llama-3.2-3B-Malaysian-Reasoning/commit/00085d8a02cd4d86ea042e24677db154d0a4200a', commit_message='Upload tokenizer', commit_description='', oid='00085d8a02cd4d86ea042e24677db154d0a4200a', pr_url=None, repo_url=RepoUrl('https://huggingface.co/mesolitica/Llama-3.2-3B-Malaysian-Reasoning', endpoint='https://huggingface.co', repo_type='model', repo_id='mesolitica/Llama-3.2-3B-Malaysian-Reasoning'), pr_revision=None, pr_num=None)

In [17]:
ori_model.push_to_hub('mesolitica/Llama-3.2-3B-Malaysian-Reasoning')

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

[2025-01-05 15:31:52,668] [INFO] [real_accelerator.py:219:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/usr/bin/ld: cannot find -laio
collect2: error: ld returned 1 exit status
/usr/bin/ld: /usr/local/cuda/lib64/libcufile.so: undefined reference to `dlopen'
/usr/bin/ld: /usr/local/cuda/lib64/libcufile.so: undefined reference to `dlclose'
/usr/bin/ld: /usr/local/cuda/lib64/libcufile.so: undefined reference to `dlerror'
/usr/bin/ld: /usr/local/cuda/lib64/libcufile.so: undefined reference to `dlsym'
collect2: error: ld returned 1 exit status


model-00002-of-00002.safetensors:   0%|          | 0.00/2.25G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

CommitInfo(commit_url='https://huggingface.co/mesolitica/Llama-3.2-3B-Malaysian-Reasoning/commit/270a323bcb5c78f31c1c3fd0569088ca5a43ae28', commit_message='Upload LlamaForCausalLM', commit_description='', oid='270a323bcb5c78f31c1c3fd0569088ca5a43ae28', pr_url=None, repo_url=RepoUrl('https://huggingface.co/mesolitica/Llama-3.2-3B-Malaysian-Reasoning', endpoint='https://huggingface.co', repo_type='model', repo_id='mesolitica/Llama-3.2-3B-Malaysian-Reasoning'), pr_revision=None, pr_num=None)